In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# path = os.getcwd()
path ='C:\\Users\\luciu\\Box Sync\\ncaa-march-madness-2020\\data\\MDataFiles_Stage1'

In [3]:
#load data
ncaa_detail = pd.read_csv(path+'\\MNCAATourneyDetailedResults.csv')
reg_detail = pd.read_csv(path+'\\MRegularSeasonDetailedResults.csv')
#second_ = pd.read_csv(path+'MSecondaryTourneyCompactResults.csv')

seed = pd.read_csv(path+'\\MNCAATourneySeeds.csv')
coach = pd.read_csv(path+'\\MTeamCoaches.csv')
ncaa_detail.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [4]:
def process(games):
    # change location to bool
    games['WLoc'] = games['WLoc'].replace("H",1)
    games['WLoc'] = games['WLoc'].replace("A",-1)
    games['WLoc'] = games['WLoc'].replace("N",0)
    # add score difference
    Score_dif = games['WScore']-games['LScore']
    Score_dif = pd.Series(Score_dif)
    games['ScoreDif'] = Score_dif
    # rebounds chance
    WRC = (games['WOR']+games['WDR'])/(games['WOR']+games['WDR']+games['LOR']+games['LDR'])
    LRC = 1-WRC
    games['WRC'] = WRC
    games['LRC'] = LRC
    # Field Score accuracy, 3 points accuracy, score ability
    WFAc = games['WFGM']/games['WFGA']
    LFAc = games['LFGM']/games['LFGA']
    WFAc3 = games['WFGM3']/games['WFGA3']
    LFAc3 = games['LFGM3']/games['LFGA3']
    WSAc = (games['WFGM']*2+games['WFGM3'])/(games['WFGA']*2+games['WFGA3'])
    LSAc = (games['LFGM']*2+games['LFGM3'])/(games['LFGA']*2+games['LFGA3'])
    games['WFAc'] = WFAc
    games['WFAc3'] = WFAc3
    games['WSAc'] = WSAc
    games['LFAc'] = LFAc
    games['LFAc3'] = LFAc3
    games['LSAc'] = LSAc
    # alter col name and order
    win_games = games[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc','NumOT',
                        'WFAc','WFAc3','WSAc','WRC','WAst', 'WStl', 'WBlk', 
                        'LFAc','LFAc3','LSAc','LRC','LAst', 'LStl', 'LBlk', 'ScoreDif']].copy()
    lose_games = games[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore','WLoc','NumOT',
                        'LFAc','LFAc3','LSAc','LRC','LAst', 'LStl', 'LBlk',
                        'WFAc','WFAc3','WSAc','WRC','WAst', 'WStl', 'WBlk',  'ScoreDif']].copy()
    lose_games.loc[:,'WLoc'] = -lose_games['WLoc']
    lose_games.loc[:,'ScoreDif'] = -lose_games['ScoreDif']
    win_games.columns = ['Season', 'DayNum', 'T1', 'T1Score', 'T2', 'T2Score', 'Loc','NumOT',
                        'T1FAc','T1FAc3','T1SAc','T1RC','T1Ast', 'T1Stl', 'T1Blk', 
                        'T2FAc','T2FAc3','T2SAc','T2RC','T2Ast', 'T2Stl', 'T2Blk', 'ScoreDif']
    lose_games.columns = ['Season', 'DayNum', 'T1', 'T1Score', 'T2', 'T2Score', 'Loc','NumOT',
                        'T1FAc','T1FAc3','T1SAc','T1RC','T1Ast', 'T1Stl', 'T1Blk', 
                        'T2FAc','T2FAc3','T2SAc','T2RC','T2Ast', 'T2Stl', 'T2Blk', 'ScoreDif']
    return pd.concat([win_games,lose_games])

In [5]:
reg = process(reg_detail)
ncaa = process(ncaa_detail)

In [6]:
# add season mean and var
reg_ = reg[reg['DayNum']>=111] # only use last three weeks
season_mean = reg_[['Season','T1','T1FAc','T1FAc3','T1SAc','T1RC','T1Ast','T1Stl','T1Blk','ScoreDif']].groupby(['Season','T1']).mean()
season_var = reg_[['Season','T1','T1FAc','T1FAc3','T1SAc','T1RC']].groupby(['Season','T1']).var()
mean_var = pd.merge(season_mean,season_var,left_on = ['T1','Season'],right_on = ['T1','Season'],how = 'left')
mean_var = mean_var.reset_index()
# copy T1 and T2
mean_var2 = mean_var.copy()
mean_var.columns = ['T1', 'Season', 'T1MFAc', 'T1MFAc3', 'T1MSAc', 'T1MRC', 'T1MAst',
       'T1MStl', 'T1MBlk', 'T1MScoreDif', 'T1VFAc', 'T1VFAc3', 'T1VSAc','T1VRC']
mean_var2.columns = ['T2', 'Season', 'T2MFAc', 'T2MFAc3', 'T2MSAc', 'T2MRC', 'T2MAst',
       'T2MStl', 'T2MBlk', 'T2MScoreDif', 'T2VFAc', 'T2VFAc3', 'T2VSAc','T2VRC']


In [7]:
# combine ncaa reg
ncaa_add1 = pd.merge(ncaa,mean_var,left_on = ['T1','Season'],right_on = ['T1','Season'],how = 'left')
ncaa_season = pd.merge(ncaa_add1,mean_var2,left_on = ['T2','Season'],right_on = ['T2','Season'],how = 'left')

In [8]:
# add seed dif
seed2 = seed.copy()
seed.columns = ['Season', 'T1Seed', 'T1']
seed2.columns = ['Season', 'T2Seed', 'T2']
all_games = pd.merge(ncaa_season,seed,left_on = ['T1','Season'], right_on = ['T1','Season'],how = 'left')
all_games = pd.merge(all_games,seed2,left_on = ['T2','Season'], right_on = ['T2','Season'],how = 'left')
all_games['T1Seed'] = all_games.T1Seed.str.extract('(\d+)').fillna(0).astype('int64')
all_games['T2Seed'] = all_games.T2Seed.str.extract('(\d+)').fillna(0).astype('int64')
all_games['SeedDif'] = all_games['T1Seed']-all_games['T2Seed']

In [9]:
all_games.columns

Index(['Season', 'DayNum', 'T1', 'T1Score', 'T2', 'T2Score', 'Loc', 'NumOT',
       'T1FAc', 'T1FAc3', 'T1SAc', 'T1RC', 'T1Ast', 'T1Stl', 'T1Blk', 'T2FAc',
       'T2FAc3', 'T2SAc', 'T2RC', 'T2Ast', 'T2Stl', 'T2Blk', 'ScoreDif',
       'T1MFAc', 'T1MFAc3', 'T1MSAc', 'T1MRC', 'T1MAst', 'T1MStl', 'T1MBlk',
       'T1MScoreDif', 'T1VFAc', 'T1VFAc3', 'T1VSAc', 'T1VRC', 'T2MFAc',
       'T2MFAc3', 'T2MSAc', 'T2MRC', 'T2MAst', 'T2MStl', 'T2MBlk',
       'T2MScoreDif', 'T2VFAc', 'T2VFAc3', 'T2VSAc', 'T2VRC', 'T1Seed',
       'T2Seed', 'SeedDif'],
      dtype='object')

In [10]:
# move output last col
# drop score
data = all_games[['Season', 'T1', 'T2','ScoreDif',
       'T1MFAc', 'T1MFAc3', 'T1MSAc', 'T1MRC', 'T1MAst', 'T1MStl', 'T1MBlk',
       'T1MScoreDif', 'T1VFAc', 'T1VFAc3', 'T1VSAc', 'T1VRC', 'T1Seed',
       'T2MFAc','T2MFAc3', 'T2MSAc', 'T2MRC', 'T2MAst', 'T2MStl', 'T2MBlk',
       'T2MScoreDif', 'T2VFAc', 'T2VFAc3', 'T2VSAc', 'T2VRC', 
       'T2Seed', 'SeedDif']].copy()

In [18]:
# data.to_csv(r'C:\\Users\\luciu\\Box Sync\\ncaa-march-madness-2020\\code\processed_pd.csv', index = False)
np.savetxt("processed_np.csv", data.values, delimiter=",")

In [19]:
## data preprocessing end

In [63]:
## prepare submission features
# read to int in dataframe
submission = pd.read_csv('C:\\Users\\luciu\\Box Sync\\ncaa-march-madness-2020\\data\\MSampleSubmissionStage1_2020.csv')
sub = np.array(submission['ID'].str.split('_').tolist()).astype(int)
sub = pd.DataFrame(data = sub)
sub.columns = ['Season','T1','T2']

In [64]:
# combine with season info
team_info = data[['Season', 'T1',
       'T1MFAc', 'T1MFAc3', 'T1MSAc', 'T1MRC', 'T1MAst', 'T1MStl', 'T1MBlk',
       'T1MScoreDif', 'T1VFAc', 'T1VFAc3', 'T1VSAc', 'T1VRC', 'T1Seed',]].copy()
team_info = team_info.groupby(['Season','T1']).mean().reset_index()
sub = pd.merge(sub,team_info,left_on = ['T1','Season'],right_on = ['T1','Season'],how = 'left')
sub = pd.merge(sub,team_info,left_on = ['T2','Season'],right_on = ['T1','Season'],how = 'left')

In [65]:
# modify columns and col_names
sub = sub.drop(['T1_y'],axis = 1)
sub.columns = ['Season', 'T1', 'T2',
       'T1MFAc', 'T1MFAc3', 'T1MSAc', 'T1MRC', 'T1MAst', 'T1MStl', 'T1MBlk',
       'T1MScoreDif', 'T1VFAc', 'T1VFAc3', 'T1VSAc', 'T1VRC', 'T1Seed',
       'T2MFAc','T2MFAc3', 'T2MSAc', 'T2MRC', 'T2MAst', 'T2MStl', 'T2MBlk',
       'T2MScoreDif', 'T2VFAc', 'T2VFAc3', 'T2VSAc', 'T2VRC', 
       'T2Seed']
sub['SeedDif'] = sub['T1Seed']-sub['T2Seed']

In [66]:
np.savetxt("submission_input.csv", sub, delimiter=",")